1\. Write a function that converts number representation (bin<->dec<->hex)

In [14]:
def conversion(number, base_1, base_2):
    if base_1 == 10:
        if base_2 == 2:
            return bin(number)
        if base_2 == 16:
            return hex(number)
    else:
        s_n = str(number)
        if base_2 == 10:
                return int(s_n,base_1)
        else:
            if base_1==2:
                if base_2 == 16:
                    return hex(int(s_n,base_1))
            if base_1==16:
                if base_2 == 2:
                    return bin(int(s_n,base_1))
    
print(conversion(1100, 2, 16))
print(conversion("c", 16, 2))
print(conversion(15, 10, 16))

0xc
0b1100
0xf


2\. Write a function that converts a 32 bit word into a single precision floating point (i.e. interprets the various bits as sign, mantissa and exponent)

In [20]:
def spfp_conversion(word):
    print(word)
    word=word[::-1]
    print(len(word))
    segno=word[31]
    e=int(word[24:31],2)
    bias=127
    esp=e-bias
    mantissa=int(word[0:24],2)
    number = (1+ mantissa*0.1)*2**esp
    if segno=='0':
        return '+'+str(number)
    else:
        return '-'+str(number)

spfp_conversion('11000000101100000000000000000000')

11000000101100000000000000000000
32


'-2.703637006891261e-38'

3\. Write a program to determine the underflow and overflow limits (within a factor of 2) for python on your computer. 

**Tips**: define two variables inizialized to 1 and halve/double them enough time to exceed the under/over-flow limits  

In [16]:
detect_under=float(1)
detect_over=float(1)
i=0
m=0
while detect_under>0:
    i +=1
    detect_under=detect_under/2
print(detect_under)
print(i)

while detect_over<float('Inf'):
    m+=1
    detect_over=detect_over*2
    
print(detect_over)
print(m)
print(2**m)

0.0
1075
inf
1024
179769313486231590772930519078902473361797697894230657273430081157732675805500963132708477322407536021120113879871393357658789768814416622492847430639474124377767893424865485276302219601246094119453082952085005768838150682342462881473913110540827237163350510684586298239947245938479716304835356329624224137216


4\. Write a program to determine the machine precision

**Tips**: define a new variable by adding a smaller and smaller value (proceeding similarly to prob. 2) to an original variable and check the point where the two are the same 

In [36]:
var=float(1)
eps=float(1)
it=0
while (var + eps) > var:
    eps=eps/2
    it +=1
print(eps)
print(it)
print(var+eps)

1.1102230246251565e-16
53
1.0


5\. Write a function that takes in input three parameters $a$, $b$ and $c$ and prints out the two solutions to the quadratic equation $ax^2+bx+c=0$ using the standard formula:
$$
x=\frac{-b\pm\sqrt{b^2-4ac}}{2a}
$$

(a) use the program to compute the solution for $a=0.001$, $b=1000$ and $c=0.001$

(b) re-express the standard solution formula by multiplying top and bottom by $-b\mp\sqrt{b^2-4ac}$ and again find the solution for $a=0.001$, $b=1000$ and $c=0.001$. How does it compare with what previously obtained? Why?

(c) write a function that compute the roots of a quadratic equation accurately in all cases

In [67]:
import math
def sol_eq(a,b,c):
    d=math.sqrt(b**2-4*a*c)
    x1=(-b+d)/(2*a)
    x2=(-b-d)/(2*a)
    return x1, x2

print(sol_eq(0.001,1000,0.001))

(-9.999894245993346e-07, -999999.999999)


In [68]:
def sol_eq_2(a,b,c):
    d=math.sqrt(b**2-4*a*c)
    x1=4*a*c/((2*a)*(-b-d))
    x2=4*a*c/((2*a)*(-b+d))
    return x1, x2

print(sol_eq_2(0.001,1000,0.001))

(-1.0000000000009999e-06, -1000010.5755125057)


In [69]:
for i in range(2):
    print(sol_eq(0.001,1000,0.001)[i]-sol_eq_2(0.001,1000,0.001)[i])

1.0575401665279555e-11
10.575513505726121


There are problems when we subtract numbers very close in value
To obtain the rigth solution we have to compute the first solution with the standard formula and the second one with the formula used in the second case presented.

In [28]:
def sol_eq_3(a,b,c):
    d=math.sqrt(b**2-4*a*c)
    x1=4*a*c/((2*a)*(-b-d))
    x2=(-b-d)/(2*a)
    
    return x1,x2
    
print(sol_eq_3(0.001,1000,0.001))

(-1.0000000000009999e-06, -999999.999999)


6\. Write a program that implements the function $f(x)=x(x−1)$

(a) Calculate the derivative of the function at the point $x = 1$ using the derivative definition:

$$
\frac{{\rm d}f}{{\rm d}x} = \lim_{\delta\to0} \frac{f(x+\delta)-f(x)}{\delta}
$$

with $\delta = 10^{−2}$. Calculate the true value of the same derivative analytically and compare with the answer your program gives. The two will not agree perfectly. Why not?

(b) Repeat the calculation for $\delta = 10^{−4}, 10^{−6}, 10^{−8}, 10^{−10}, 10^{−12}$ and $10^{−14}$. How does the accuracy scales with $\delta$?

In [70]:
def f(x):
    return x*(x-1)

In [71]:
def der(x,d):
    delta=d
    return (f(x+delta)-f(x))/delta

def der2(x):
    return 2*x-1

print('derivative, delta=10^-2:',der2(1))

for i in range(1,8):
    print('derivative with definition, delta=10^-',2*i,':',der(1, 10**(-(2*i))))

derivative, delta=10^-2: 1
derivative with definition, delta=10^- 2 : 1.010000000000001
derivative with definition, delta=10^- 4 : 1.0000999999998899
derivative with definition, delta=10^- 6 : 1.0000009999177333
derivative with definition, delta=10^- 8 : 1.0000000039225287
derivative with definition, delta=10^- 10 : 1.000000082840371
derivative with definition, delta=10^- 12 : 1.0000889005833413
derivative with definition, delta=10^- 14 : 0.9992007221626509


7\. Consider the integral of the semicircle of radius 1:
$$
I=\int_{-1}^{1} \sqrt(1-x^2) {\rm d}x
$$
which it's known to be $I=\frac{\pi}{2}=1.57079632679...$.
Alternatively we can use the Riemann definition of the integral:
$$
I=\lim_{N\to\infty} \sum_{k=1}^{N} h y_k 
$$

with $h=2/N$ the width of each of the $N$ slices the domain is divided into and where
$y_k$ is the value of the function at the $k-$th slice.

(a) Write a programe to compute the integral with $N=100$. How does the result compares to the true value?

(b) How much can $N$ be increased if the computation needs to be run in less than a second? What is the gain in running it for 1 minute? 


In [57]:
import math
def semicircle(x):
    return math.sqrt(1-x**2)

def integral(n):
    N = n
    h = 2/N
    I=0
    true_I=1.57079632679
    value=-1
    for i in range(1,N+1):
        I+=h*semicircle(value)
        value+=h
    return (I,true_I-I)

N=100

I, I_comparison = integral(N)
print("integral computation: ",I)
print("inegral compared: ",true_I-I)




integral computation:  1.5691342555492493
inegral compared:  0.0016620712407506844


In [62]:
N=100
for i in range(7):
    print('N:',N,'result:',integral(N))
    %timeit integral(N)
    N = N*10
    print()

N: 100 result: (1.5691342555492493, 0.0016620712407506844)
50.4 µs ± 2.79 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)

N: 1000 result: (1.5707437385010703, 5.258828892973355e-05)
510 µs ± 22.3 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)

N: 10000 result: (1.5707946637153898, 1.6630746102475058e-06)
5.19 ms ± 206 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)

N: 100000 result: (1.57079627420242, 5.258758006831954e-08)
52.4 ms ± 3.9 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)

N: 1000000 result: (1.570796325147268, 1.642731950468601e-09)
496 ms ± 11 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

N: 10000000 result: (1.5707963267012999, 8.870015832940226e-11)
5.11 s ± 146 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

N: 100000000 result: (1.570796327457729, -6.677289832168753e-10)
54.3 s ± 3.12 s per loop (mean ± std. dev. of 7 runs, 1 loop each)

